# Coflect HITL Module Quickstart\n\nThis notebook shows a practical local flow for the `hitl` module using Torch backend.

## 1) Verify package installation\n\nInstall first (outside notebook):\n\n```bash\npip install -e .[server,dev]\n```

In [ ]:
import coflect\nfrom coflect.modules import list_modules\nfrom coflect.backends import list_backends\n\nprint('coflect version:', coflect.__version__)\nprint('registered modules:', list_modules())\nprint('registered backends:', list_backends())

## 2) Launch processes\n\nRun these commands in separate terminals.

In [ ]:
backend_cmd = 'coflect-hitl-backend --host 0.0.0.0 --port 8000'\ntrainer_cmd = 'coflect-hitl-trainer-torch --server http://localhost:8000 --steps 1000 --xai_every 100 --forecast_every 20'\nforecast_cmd = 'coflect-hitl-forecast-worker --server http://localhost:8000 --backend torch'\nworker_cmd = 'coflect-hitl-xai-worker-torch --server http://localhost:8000 --xai_method consensus'\nui_url = 'http://localhost:8000'\n\nprint('1) ', backend_cmd)\nprint('2) ', trainer_cmd)\nprint('3) ', forecast_cmd)\nprint('4) ', worker_cmd)\nprint('5) Open browser:', ui_url)

## 3) Send feedback (ROI-guided)\n\nThis simulates user guidance from code; UI supports ROI drawing as well.

In [ ]:
import requests\n\npayload = {\n    'step': 100,\n    'sample_idx': 42,\n    'instruction': 'Focus on center region',\n    'strength': 0.7,\n    'roi': {'x0': 0.30, 'y0': 0.30, 'x1': 0.70, 'y1': 0.70},\n}\n\npost_resp = requests.post('http://localhost:8000/feedback', json=payload, timeout=5)\nprint('POST /feedback:', post_resp.status_code, post_resp.json())\n\nget_resp = requests.get('http://localhost:8000/feedback', timeout=5)\nprint('GET /feedback:', get_resp.status_code, get_resp.json())

## 4) Validate in UI\n\nOpen `http://localhost:8000` and confirm:\n- Metrics stream updates.\n- Forecast top-10 queue updates when review window is open.\n- LiveCAM overlays update for selected forecast samples.\n- Feedback affects `focus_lambda` without stalling trainer.